<a href="https://colab.research.google.com/github/DoItSon/Dacon/blob/main/%EA%B1%B4%EC%84%A4%EA%B8%B0%EA%B3%84%20%EC%98%A4%EC%9D%BC%20%EC%83%81%ED%83%9C%20%EB%B6%84%EB%A5%98%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/CAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import random

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

from lightgbm import lgb

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.metrics import explained_variance_score, r2_score, mean_absolute_error

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 6.5 MB/s 
     |████████████████████████████████| 209 kB 64.6 MB/s 
     |████████████████████████████████| 81 kB 3.0 MB/s 
     |████████████████████████████████| 78 kB 4.0 MB/s 
     |████████████████████████████████| 147 kB 23.3 MB/s 
     |████████████████████████████████| 50 kB 3.4 MB/s 
     |████████████████████████████████| 112 kB 19.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=0c7d5b705b9aac7d9f138dffa91fee6127a28a56c050a11712e4c337f977fe39
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [ ]:
import optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/"

In [ ]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

In [ ]:
categorical_features_enc = ['COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features_enc = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

In [ ]:
CFG = {
    'n_splits':5,
    'SEED':42
}

## 전처리

In [ ]:
cols = ["CD","FH2O","FNOX","FOPTIMETHGLY","FOXID","FSO4","FTBN","FUEL","K","SOOTPERCENTAGE","U100","U75","U50","U25","U20","U14","U6","U4","V100"]

In [ ]:
imputer = IterativeImputer(estimator =  lgb.LGBMRegressor(),random_state=42)
tmp = imputer.fit_transform(train[cols])

In [ ]:
x2 = pd.DataFrame(tmp,columns=cols)
train = train.drop(columns=cols)
train = pd.concat([train,x2],axis=1)
train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,SOOTPERCENTAGE,U100,U75,U50,U25,U20,U14,U6,U4,V100
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,1.136802,0.980376,1.043866,42.034095,53.104079,63.133355,-56.375867,-5350.870144,82968.454117,12.629231
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,1.285350,0.000000,0.000000,1.000000,2.000000,4.000000,6.000000,216.000000,1454.000000,12.628924
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,1.096286,0.000000,0.000000,0.000000,0.000000,3.000000,39.000000,11261.000000,41081.000000,12.673944
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,0.744939,1.023447,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,1.035179,0.001170,1.000669,2.743480,36.845274,60.082252,274.569830,3754.328873,9619.020023,12.406262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,...,0.879413,-0.000468,0.315118,2.978744,23.778101,62.527363,512.705479,12676.031707,14307.650019,12.743101
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,...,0.400000,1.737289,9.305085,26.398081,58.780116,27.172921,1989.173172,31155.412941,40438.425366,14.500000
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,...,0.744939,1.023447,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,...,1.248462,0.000000,0.000000,0.000000,7.000000,8.000000,100.000000,1625.000000,18890.000000,12.196332


In [ ]:
train.isnull().sum().sum()

## 데이터 분리

In [ ]:
train_X  = train.drop(['ID', 'Y_LABEL'], axis = 1) # 정답값 제외
target  = train['Y_LABEL'] # 정답

test = test.drop(['ID'], axis = 1) # 추론 데이터

In [ ]:
train_X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,SOOTPERCENTAGE,U100,U75,U50,U25,U20,U14,U6,U4,V100
0,COMPONENT3,1486,2011,7,200,0,3,93,0,0,...,1.136802,0.980376,1.043866,42.034095,53.104079,63.133355,-56.375867,-5350.870144,82968.454117,12.629231
1,COMPONENT2,1350,2021,51,375,0,2,19,0,0,...,1.285350,0.000000,0.000000,1.000000,2.000000,4.000000,6.000000,216.000000,1454.000000,12.628924
2,COMPONENT2,2415,2015,2,200,0,110,1,1,0,...,1.096286,0.000000,0.000000,0.000000,0.000000,3.000000,39.000000,11261.000000,41081.000000,12.673944
3,COMPONENT3,7389,2010,2,200,0,8,3,0,0,...,0.744939,1.023447,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169
4,COMPONENT3,3954,2015,4,200,0,1,157,0,0,...,1.035179,0.001170,1.000669,2.743480,36.845274,60.082252,274.569830,3754.328873,9619.020023,12.406262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,...,0.879413,-0.000468,0.315118,2.978744,23.778101,62.527363,512.705479,12676.031707,14307.650019,12.743101
14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,...,0.400000,1.737289,9.305085,26.398081,58.780116,27.172921,1989.173172,31155.412941,40438.425366,14.500000
14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,...,0.744939,1.023447,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169
14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,...,1.248462,0.000000,0.000000,0.000000,7.000000,8.000000,100.000000,1625.000000,18890.000000,12.196332


## 피처 추가

In [ ]:
features = train_X[["FNOX","FH2O"]].mean(axis=1).reset_index()
features.columns = ["Index","FNOX+FH2O"]

x = pd.concat([train_X,features],axis = 1)
train_X= x.drop(columns=["Index"])
train_X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,U100,U75,U50,U25,U20,U14,U6,U4,V100,FNOX+FH2O
0,COMPONENT3,1486,2011,7,200,0,3,93,0,0,...,0.980376,1.043866,42.034095,53.104079,63.133355,-56.375867,-5350.870144,82968.454117,12.629231,8.770214
1,COMPONENT2,1350,2021,51,375,0,2,19,0,0,...,0.000000,0.000000,1.000000,2.000000,4.000000,6.000000,216.000000,1454.000000,12.628924,11.448895
2,COMPONENT2,2415,2015,2,200,0,110,1,1,0,...,0.000000,0.000000,0.000000,0.000000,3.000000,39.000000,11261.000000,41081.000000,12.673944,11.413547
3,COMPONENT3,7389,2010,2,200,0,8,3,0,0,...,1.023447,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169,9.094943
4,COMPONENT3,3954,2015,4,200,0,1,157,0,0,...,0.001170,1.000669,2.743480,36.845274,60.082252,274.569830,3754.328873,9619.020023,12.406262,8.871371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,...,-0.000468,0.315118,2.978744,23.778101,62.527363,512.705479,12676.031707,14307.650019,12.743101,8.961282
14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,...,1.737289,9.305085,26.398081,58.780116,27.172921,1989.173172,31155.412941,40438.425366,14.500000,10.500000
14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,...,1.023447,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169,9.094943
14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,...,0.000000,0.000000,0.000000,7.000000,8.000000,100.000000,1625.000000,18890.000000,12.196332,11.108078


In [ ]:
features = train_X[["FNOX","FOXID"]].mean(axis=1).reset_index()
features.columns = ["Index","FNOX+FOXID"]

x = pd.concat([train_X,features],axis = 1)
train_X= x.drop(columns=["Index"])
train_X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,U75,U50,U25,U20,U14,U6,U4,V100,FNOX+FH2O,FNOX+FOXID
0,COMPONENT3,1486,2011,7,200,0,3,93,0,0,...,1.043866,42.034095,53.104079,63.133355,-56.375867,-5350.870144,82968.454117,12.629231,8.770214,8.313554
1,COMPONENT2,1350,2021,51,375,0,2,19,0,0,...,0.000000,1.000000,2.000000,4.000000,6.000000,216.000000,1454.000000,12.628924,11.448895,11.846022
2,COMPONENT2,2415,2015,2,200,0,110,1,1,0,...,0.000000,0.000000,0.000000,3.000000,39.000000,11261.000000,41081.000000,12.673944,11.413547,12.142620
3,COMPONENT3,7389,2010,2,200,0,8,3,0,0,...,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169,9.094943,9.690969
4,COMPONENT3,3954,2015,4,200,0,1,157,0,0,...,1.000669,2.743480,36.845274,60.082252,274.569830,3754.328873,9619.020023,12.406262,8.871371,9.214363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,...,0.315118,2.978744,23.778101,62.527363,512.705479,12676.031707,14307.650019,12.743101,8.961282,9.494610
14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,...,9.305085,26.398081,58.780116,27.172921,1989.173172,31155.412941,40438.425366,14.500000,10.500000,12.000000
14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,...,2.066748,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169,9.094943,9.690969
14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,...,0.000000,0.000000,7.000000,8.000000,100.000000,1625.000000,18890.000000,12.196332,11.108078,11.631320


In [ ]:
features = train_X[["V100","V40"]].mean(axis=1).reset_index()
features.columns = ["Index","V100+V40"]

x = pd.concat([train_X,features],axis = 1)
train_X= x.drop(columns=["Index"])
train_X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,U50,U25,U20,U14,U6,U4,V100,FNOX+FH2O,FNOX+FOXID,V100+V40
0,COMPONENT3,1486,2011,7,200,0,3,93,0,0,...,42.034095,53.104079,63.133355,-56.375867,-5350.870144,82968.454117,12.629231,8.770214,8.313554,83.314615
1,COMPONENT2,1350,2021,51,375,0,2,19,0,0,...,1.000000,2.000000,4.000000,6.000000,216.000000,1454.000000,12.628924,11.448895,11.846022,28.314462
2,COMPONENT2,2415,2015,2,200,0,110,1,1,0,...,0.000000,0.000000,3.000000,39.000000,11261.000000,41081.000000,12.673944,11.413547,12.142620,42.636972
3,COMPONENT3,7389,2010,2,200,0,8,3,0,0,...,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169,9.094943,9.690969,73.466584
4,COMPONENT3,3954,2015,4,200,0,1,157,0,0,...,2.743480,36.845274,60.082252,274.569830,3754.328873,9619.020023,12.406262,8.871371,9.214363,72.753131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,...,2.978744,23.778101,62.527363,512.705479,12676.031707,14307.650019,12.743101,8.961282,9.494610,74.071551
14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,...,26.398081,58.780116,27.172921,1989.173172,31155.412941,40438.425366,14.500000,10.500000,12.000000,66.000000
14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,...,3.406391,27.369519,40.140005,345.366183,21854.737778,36280.937074,13.633169,9.094943,9.690969,33.816584
14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,...,0.000000,7.000000,8.000000,100.000000,1625.000000,18890.000000,12.196332,11.108078,11.631320,28.248166


## Classifier

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [ ]:
import catboost as cb

In [ ]:
cat_features = ['COMPONENT_ARBITRARY']

In [ ]:
score_list = []

model = cb.CatBoostClassifier(random_state=CFG["SEED"],cat_features = cat_features, one_hot_max_size=4) #,task_type="GPU" 
model.fit(train_X, target)

pred = model.predict_proba(train_X)[:,1]
# predict = np.where(pred > 0.35, 1 , 0)
score = cross_val_score(model, train_X, target, scoring="f1_macro")
score_list.append(score) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 0.5247624	total: 63.9ms	remaining: 12.7s
5:	learn: 0.4983310	total: 75.4ms	remaining: 12.5s
6:	learn: 0.4726534	total: 87.2ms	remaining: 12.4s
7:	learn: 0.4503906	total: 101ms	remaining: 12.5s
8:	learn: 0.4296755	total: 122ms	remaining: 13.4s
9:	learn: 0.4110123	total: 133ms	remaining: 13.2s
10:	learn: 0.3935234	total: 145ms	remaining: 13.1s
11:	learn: 0.3780357	total: 165ms	remaining: 13.6s
12:	learn: 0.3636475	total: 177ms	remaining: 13.4s
13:	learn: 0.3503416	total: 189ms	remaining: 13.3s
14:	learn: 0.3373217	total: 204ms	remaining: 13.4s
15:	learn: 0.3259711	total: 215ms	remaining: 13.3s
16:	learn: 0.3146717	total: 228ms	remaining: 13.2s
17:	learn: 0.3047488	total: 239ms	remaining: 13.1s
18:	learn: 0.2955465	total: 251ms	remaining: 13s
19:	learn: 0.2859124	total: 263ms	remaining: 12.9s
20:	learn: 0.2780015	total: 276ms	remaining: 12.8s
21:	learn: 0.2710009	total: 287ms	remaining: 12.8s
22:	learn: 0.2647613	total: 298ms	remaining: 12.7s


In [ ]:
np.mean(score)

0.8192507952588167

- 확률 추출

In [ ]:
target_X = pred
target_X

array([0.0097608 , 0.00477487, 0.98083806, ..., 0.21259504, 0.01714909,
       0.00183511])

In [ ]:
test_stage_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY','YEAR']

In [ ]:
train_reg = train_X[test_stage_features]
train_reg.shape,target_X.shape

((14095, 18), (14095,))

# 원핫인코딩
- test데이터를 cat_features에 넣는 방법을 모르겠다.

In [ ]:
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(train_reg[["COMPONENT_ARBITRARY"]]).toarray(),
    columns = enc.get_feature_names_out()
)
train_reg = pd.concat([train_reg,tmp],axis=1).drop(columns = ["COMPONENT_ARBITRARY"])

tmp1 = pd.DataFrame(
    enc.transform(test[["COMPONENT_ARBITRARY"]]).toarray(),
    columns = enc.get_feature_names_out()
)
test = pd.concat([test,tmp1],axis=1).drop(columns = ["COMPONENT_ARBITRARY"])

In [ ]:
score_list = []

model = cb.CatBoostRegressor(random_state=CFG["SEED"]) # ,task_type="GPU"
model.fit(train_reg, target_X)
pred =  model.predict(train_reg)

score = -1 * cross_val_score(model, train_reg, target_X, scoring="neg_mean_absolute_error")
score_list.append(score) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 0.2063845	total: 36.8ms	remaining: 7.32s
5:	learn: 0.2059150	total: 42.6ms	remaining: 7.06s
6:	learn: 0.2053946	total: 46.5ms	remaining: 6.59s
7:	learn: 0.2050086	total: 52.5ms	remaining: 6.5s
8:	learn: 0.2046614	total: 56.2ms	remaining: 6.19s
9:	learn: 0.2042030	total: 61ms	remaining: 6.04s
10:	learn: 0.2038704	total: 69.4ms	remaining: 6.24s
11:	learn: 0.2034205	total: 82.1ms	remaining: 6.76s
12:	learn: 0.2030345	total: 95.7ms	remaining: 7.27s
13:	learn: 0.2027626	total: 120ms	remaining: 8.42s
14:	learn: 0.2025659	total: 132ms	remaining: 8.64s
15:	learn: 0.2022447	total: 142ms	remaining: 8.76s
16:	learn: 0.2019897	total: 151ms	remaining: 8.74s
17:	learn: 0.2018655	total: 160ms	remaining: 8.73s
18:	learn: 0.2017252	total: 170ms	remaining: 8.78s
19:	learn: 0.2016149	total: 183ms	remaining: 8.99s
20:	learn: 0.2014581	total: 196ms	remaining: 9.13s
21:	learn: 0.2012395	total: 206ms	remaining: 9.14s
22:	learn: 0.2011038	total: 220ms	remaining: 9

In [ ]:
np.mean(score_list)

0.09873571154647273

In [ ]:
pred = model.predict(test)

In [ ]:
np.save(f"{DATA_PATH}/CAT",pred)

In [ ]:
y_pred = pred

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
submit.Y_LABEL = pd.Series(np.where(y_pred > 0.2, 1, 0))

In [ ]:
submit

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,1
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [ ]:
submit.Y_LABEL.value_counts()

0    5675
1     366
Name: Y_LABEL, dtype: int64

In [ ]:
submit.to_csv('/content/drive/MyDrive/submit.csv', index=False)